In [42]:
import pandas as pd
from os import listdir
from os.path import isfile, join
from bs4 import BeautifulSoup
import requests

In [43]:
# Extract "Article Name" from all "Article Name.html" for a list of article names
file_names = [name[:-5] for name in listdir('../wikiarticles') if isfile(join('../wikiarticles', name))]
file_names.remove(".DS_")
# turn list into df
wikiarticles = pd.DataFrame(file_names,columns=['article_name'])
wikiarticles = wikiarticles.set_index('article_name') #set article name as index

In [6]:
def wiki_url_firstdate_extractor(filename):
    '''
    Program to extract wgPageName from html of each site, then returns url
    Looking through html file to get index for wgPageName in html
    Then, iterating through the strings after to locate the next ", where the wgPageName ends
    Finally, returns the functional wikipedia url that redirects to the history page
    when article was first created
    '''
    html_handle = open(filename)
    html_text = html_handle.read()
    wgPageName_index = html_text.find('"wgPageName":')

    # for loop to find the second quotation mark marking the end of wgPageName
    end_index = 0
    for i in range(wgPageName_index + 14, wgPageName_index + 500):
        if html_text[i] == '"':
            end_index= i
            break

    wgPageName_raw = html_text[wgPageName_index+14:end_index]
    wgPageName = wgPageName_raw.replace("'", "%27") #replace ' with %27
    
    url = "https://en.wikipedia.org/w/index.php?title=" + wgPageName + "&dir=prev&action=history"
    return url

In [9]:
# For each article, there's a history page.
# This loop goes to the first date of article's creation and gets the value
# Add resulting column (full of article creation dates) to dataframe
create_date = {}
for article_name in wikiarticles.index:
    url = wiki_url_firstdate_extractor('../wikiarticles/' + article_name + '.html')
    temp = requests.get(url)
    soup = BeautifulSoup(temp.text)
    try: #on some articles I'm getting an error
        date = soup.find_all('a', attrs={"class":"mw-changeslist-date"})[-1].get_text()
        create_date[date] = [article_name]
    except: 
        pass

ValueError: Length of values does not match length of index

In [49]:
# Out of ~1600 articles, 3 articles threw an error. I'm manually adding those dates
list_of_missing = []
for article in wikiarticles.index:
    if article not in str(create_date.values()):
        list_of_missing.append(article)
create_date['00:27, 5 November 2003‎'] = ['Max &amp; Ruby']
create_date['14:54, 8 April 2019'] = ['Admiralty Powers, &amp;c. Act 1865']
create_date['20:54, 15 December 2005‎'] = ['Abraham &amp; Straus']

# add dictionary to create new column in master df
wikiarticles['date_article_creation'] = create_date

In [ ]:
# Save current dataframe to csv for next step
